# 1. About simulation of models and trajectories

The goal of this notebook is to present the basics of the simulation in our package.

Let's get familiar with the package. First, we shoud load it.

In [2]:
using MarkovProcesses

┌ Info: Precompiling MarkovProcesses [top-level]
└ @ Base loading.jl:1260


## Models

In this package, we are focused on Continuous-Time Markov Chains models that can be described by Chemical Reaction Networks.

Let's simulate our first model. We consider the famous SIR epidemiology model described by two reactions:

$$
Infection: S + I \xrightarrow{k_i} 2I \\
Recovery: I \xrightarrow{k_r} R
$$

In the MarkovProcesses package, models are objects that can be instantiatied and their types all derived from the abstract type `Model`. Let's load the SIR model.

In [3]:
load_model("SIR")

This function searchs a model file called SIR.jl in the models/ directory (at the root of the package). This files creates the necessary resources to create a `ContinuousTimeModel` and store it in a variable called SIR.

In [6]:
println(SIR)

SIR SSA pkg model (ContinuousTimeModel)
- variables :
* I (index = 2 in state space)
* R (index = 3 in state space)
* S (index = 1 in state space)
- parameters :
* ki (index = 1 in parameter space)
* kr (index = 2 in parameter space)
- transitions : R1,R2
- observed variables :
* I (index = 1 in observed state space, index = 2 in state space)
p = [0.02, 0.07]
x0 = [95, 5, 0]
t0 = 0.0
time bound = Inf


This variable contains a parameter vector and an initial point. It is ready to use.

In [4]:
@show SIR.p
@show SIR.x0

SIR.p = [0.0012, 0.05]
SIR.x0 = [95, 5, 0]


3-element Array{Int64,1}:
 95
  5
  0

You can change the parameters with the function `set_param!`

In [5]:
set_param!(SIR, :ki, 0.015)
@show SIR.p
set_param!(SIR, [0.02, 0.07])
@show SIR.p

SIR.p = [0.015, 0.05]
SIR.p = [0.02, 0.07]


2-element Array{Float64,1}:
 0.02
 0.07

## Trajectories

The simulation of the model is done by the function `simulate`.

In [7]:
σ = simulate(SIR)

Trajectory
- Model name: SIR SSA pkg 
- Variable trajectories:
* I: [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 46, 47, 48, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 79, 78, 79, 80, 81, 82, 83, 82, 81, 82, 83, 84, 85, 86, 87, 86, 87, 86, 87, 88, 87, 86, 85, 84, 85, 86, 87, 86, 85, 84, 83, 82, 81, 80, 79, 78, 77, 76, 75, 74, 73, 72, 71, 70, 69, 68, 67, 66, 65, 64, 63, 62, 61, 60, 59, 58, 57, 56, 55, 54, 53, 52, 51, 50, 49, 48, 47, 46, 45, 44, 43, 42, 41, 40, 39, 38, 37, 36, 35, 34, 33, 32, 31, 30, 29, 28, 27, 26, 25, 24, 23, 22, 21, 20, 19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0]
- times  = [0.0, 0.24577855621552194, 0.2478149161565589, 0.2853150202535163, 0.4573465669300486, 0.5156802507122017, 0.5488713853151262, 0.5694106885211497, 0.57

`simulate` returns a path which type is derived from `AbstractTrajectory`. It can be either an object of type `Trajectory` for models `::ContinuousTimeModel` or `SynchronizedTrajectory` for models that includes an automaton (but this is the subject of another notebook). It is easy to access the values of a trajectory:

In [8]:
@show σ[3] # the third state of the trajectory
@show length_states(σ) # number of states
@show σ[:I, 4] # Fourth value of the variable I
@show σ.I[4] # Fourth value of the variable I
@show get_state_from_time(σ, 2.3)

σ[3] = [7]
length_states(σ) = 196
σ[:I, 4] = 8
σ.I[4] = 8
get_state_from_time(σ, 2.3) = [83]


1-element Array{Int64,1}:
 83

The SIR object includes an observation model symbolized by the vector `SIR.g`. Even if the variables of the model are ["S", "I", "R"], only "I" will be observed.

In [9]:
@show SIR.map_var_idx
@show SIR.g
@show size(σ.values), length(σ[:I]) # Only one column which corresponds to the I variable

SIR.map_var_idx = Dict(:I => 2,:R => 3,:S => 1)
SIR.g = [:I]
(size(σ.values), length(σ[:I])) = ((1,), 196)


((1,), 196)

The SIR model is by default unbounded, i.e. each trajectory is simulated until it reaches an absorbing state.

In [10]:
@show isbounded(SIR)
@show isbounded(σ)

isbounded(SIR) = false
isbounded(σ) = false


false

For example, computations of Lp integral distances needs bounded trajectories.

In [11]:
dist_lp(simulate(SIR),simulate(SIR); p=2)

┌ Warning: Lp distance computed on unbounded trajectories. Result should be wrong
└ @ MarkovProcesses /home/moud/MarkovProcesses/markovprocesses.jl/core/trajectory.jl:61


31.26273873394255

But this feature can be changed.

In [12]:
set_time_bound!(SIR, 120.0)

120.0

In [13]:
@show dist_lp(simulate(SIR),simulate(SIR))
@show isbounded(SIR)
@show isbounded(simulate(SIR))

dist_lp(simulate(SIR), simulate(SIR)) = 164.33150971327115
isbounded(SIR) = true
isbounded(simulate(SIR)) = true


true

## Buffer size

A useful feature is that we can change the preallocation of the memory size during the simulation in order to gain computational performance.

In [18]:
load_model("ER")
@show ER.buffer_size
@timev σ = simulate(ER)
@show length_states(σ)

ER.buffer_size = 10
  0.036003 seconds (54.02 k allocations: 2.972 MiB)
elapsed time (ns): 36002718
bytes allocated:   3115887
pool allocs:       53998
non-pool GC allocs:18
length_states(σ) = 385


385

By default, the buffer size is 10. It means that a matrix of size 10 is allocated anf filled. When it is full, another matrix of size 10 is allocated. But if you know that your simulations will have a certain number of states, you can change the buffer size. It can make a big difference in terms of performance.

In [19]:
ER.buffer_size = 100
@timev σ2 = simulate(ER)
@show length_states(σ2)

  0.000154 seconds (411 allocations: 59.938 KiB)
elapsed time (ns): 153854
bytes allocated:   61376
pool allocs:       399
non-pool GC allocs:12
length_states(σ2) = 379


379

## Plots